In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import re
import os.path
from os import path

In [ ]:
driver = webdriver.Chrome(executable_path='C:\chromedriver\chromedriver.exe')

In [ ]:
# Get number of matches from HLTV given full content and 1 star fitler
driver.get('https://www.hltv.org/results?content=stats&stars=1')
content = driver.page_source
soup = BeautifulSoup(content)
n = int(soup.find('span', {'class':'pagination-data'}).text.split()[-1])
offset = range(0, n, 100)

In [ ]:
if not path.exists('matchIDs.csv'):

    matchIDs = pd.DataFrame(columns=['Date', 'ID', 'Status'])

    for page in offset:

        driver.get('https://www.hltv.org/results?offset=' + str(page) + '&content=stats&stars=1')
        content = driver.page_source
        soup = BeautifulSoup(content)
        contentpage = soup.find('div', {'class':'results-holder allres'})

        for subset in contentpage.findAll('div', {'class':'results-sublist'}):
            date = subset.find('div', {'class':'standard-headline'}).text
            date = date[12:]

            for URL in subset.findAll('a', {'class':'a-reset'}):
                ID = URL['href'].split('/')[2]
                matchIDs = matchIDs.append({'Date':date, 'ID':int(ID), 'Status':0}, ignore_index = True)

        print(page)
        sleeptime = random.uniform(1, 2)
        time.sleep(sleeptime)

    matchIDs = matchIDs[::-1].reset_index(drop = True)
    matchIDs.to_csv('matchIDs.csv', index = False)

else:
    print('File already exists')

In [ ]:
matchIDs = pd.read_csv('matchIDs.csv')

In [ ]:
# Run test
matchIDs = matchIDs[:-148]

In [ ]:
if path.exists('matchIDs.csv'):

    harvest = pd.DataFrame()

    for page in offset:

        driver.get('https://www.hltv.org/results?offset=' + str(page) + '&content=stats&stars=1')
        content = driver.page_source
        soup = BeautifulSoup(content)
        contentpage = soup.find('div', {'class':'results-holder allres'})
        batch = pd.DataFrame(columns=['Date', 'ID', 'Status'])

        for subset in contentpage.findAll('div', {'class':'results-sublist'}):
            date = subset.find('div', {'class':'standard-headline'}).text
            date = date[12:]

            for URL in subset.findAll('a', {'class':'a-reset'}):
                ID = URL['href'].split('/')[2]
                batch = batch.append({'Date':date, 'ID':int(ID), 'Status':0}, ignore_index = True)
        
        booler = batch['ID'].isin(matchIDs['ID'])
        batch = batch[~booler]
        if sum(booler) == 100:
            print('Break')
            break

        harvest = harvest.append(batch)
        print(page)
        sleeptime = random.uniform(1, 2)
        time.sleep(sleeptime)

    harvest = harvest[::-1]
    matchIDs = matchIDs.append(harvest).reset_index(drop = True)
    print(str(len(harvest)) + ' new matches have been found and added')

else:
    print('File does not exist, initialize first')

In [ ]:
# Flag status when scraped
for row in matchIDs.itertuples():
    print(row.Date, row.ID, row.Status)
    matchIDs.loc[row.Index, 'Status'] = 1

In [ ]:
testIDs = matchIDs[:5]
print(testIDs)

In [ ]:
ID = 2298365 

In [ ]:
driver.get('https://www.hltv.org/matches/' + str(ID) +'/anything')
content = driver.page_source
soup = BeautifulSoup(content)

In [ ]:
matchdata  = {}
matchdata['Date'] = 'October 1st 2015'
matchdata['Event'] = soup.find('div', {'class':'event text-ellipsis'}).text

In [ ]:
team1info = soup.find('div', {'class': 'team1-gradient'})
team1name = team1info.find('div', {'class':'teamName'}).text
team1score = team1info.find('div', {'class':['won', 'lost', 'tie']}).text

team2info = soup.find('div', {'class': 'team2-gradient'})
team2name = team2info.find('div', {'class':'teamName'}).text
team2score = team2info.find('div', {'class':['won', 'lost', 'tie']}).text

teams = [team1name, team2name]
score = [team1score, team2score]
print(teams)
print(score)

In [ ]:
# Maps and Odds